In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import seaborn as sns
import sys
import copy
sys.path.append("..")
from tqdm.notebook import tqdm
from numba import jit
from scipy import stats
import networkx as nx


import warnings
warnings.filterwarnings('ignore')
plt.style.use("../config/custom_plt.mplstyle")

colors = [
    "#7494d3",
    "#5cb545",
    "#9956c6",
    "#a7b338",
    "#6a6bc6",
    "#d09e40",
    "#ce62bb",
    "#56be85",
    "#d1477d",
    "#397f4d",
    "#cf4b4a",
    "#40bbc1",
    "#d8662c",
    "#99af66",
    "#b76989",
    "#6d7127",
    "#b6744a"
]

In [2]:
from sklearn.preprocessing import LabelEncoder
import networkx.algorithms.community as nx_comm

def load_results(inf_coords_path, labels, g):
    inf_coords = pd.read_csv(inf_coords_path, comment="#", header=None, sep="\s+")
    inf_coords.columns = ['index', 'kappa', 'hyp_rad', 'p1', 'p2', 'p3']
    inf_coords['index'] = inf_coords['index'].astype(str)
    inf_coords = inf_coords.merge(labels, on="index")
    le = LabelEncoder()
    inf_coords['encoded_label'] = le.fit_transform(inf_coords['label'])
    inf_coords = inf_coords.drop_duplicates(subset=['index'])
        
    # Louvain communities
    communities = nx_comm.louvain_communities(g, seed=123)
    communities_dict = []
    for i, com in enumerate(communities):
        communities_dict.append({c:i for c in com})

    result = {}
    for d in communities_dict:
        result.update(d)

    communities_louvain = pd.DataFrame()
    communities_louvain['index'] = result.keys()
    communities_louvain['label_louvain'] = result.values()

    inf_coords = inf_coords.merge(communities_louvain, on='index')        
    print('Number of communities from Louvain: ', len(np.unique(inf_coords['label_louvain'])))

    return inf_coords

In [3]:
texas_graph = nx.read_edgelist("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/f_mercator/webkb/texas/out1_graph_edges.edge")
texas_edges = nx.to_pandas_edgelist(texas_graph)
texas_features = pd.read_csv("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/f_mercator/webkb/texas/out1_node_feature_label.txt", sep="\s+")
texas_labels = pd.DataFrame()
texas_labels['index'] = texas_features['node_id']
texas_labels['label'] = texas_features['label']
texas_labels['index'] = texas_labels['index'].astype(str)

base_path = "/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/f_mercator/webkb/texas/umap/"
texas_le_ml = load_results(f"{base_path}/le_ml/out1_graph_edges.inf_coord", texas_labels, texas_graph)
texas_umap_ml = load_results(f"{base_path}/umap_ml/out1_graph_edges.inf_coord", texas_labels, texas_graph)
texas_only_umap = load_results(f"{base_path}/only_umap/out1_graph_edges.inf_coord", texas_labels, texas_graph)

texas_find_k = pd.read_csv("/home/rob/MEGAsync/datasets/networks/machine_learning_datasets/f_mercator/texas_labels_umap_find_k_cC.csv")
texas_find_k['index'] = texas_find_k['index'].astype(str)

texas_le_ml = texas_le_ml.merge(texas_find_k)
texas_umap_ml = texas_umap_ml.merge(texas_find_k)
texas_only_umap = texas_only_umap.merge(texas_find_k)


Number of communities from Louvain:  12
Number of communities from Louvain:  12
Number of communities from Louvain:  12


In [4]:
texas_le_ml

,index,kappa,hyp_rad,p1,p2,p3,label,encoded_label,label_louvain,label_clustering_find_k
0,0,1.836260,8.96521,3.075040,-2.258660,-0.072640,3,3,6,3
1,1,0.614698,10.05960,0.094975,-3.813790,-0.092874,0,0,9,0
2,2,0.510288,10.24570,2.464120,-2.913090,0.068231,2,2,3,1
3,3,0.911112,9.66603,2.925250,0.846579,-2.299760,3,3,1,2
4,4,2.748900,8.56174,0.089958,-3.783370,0.490599,4,4,9,0
...,...,...,...,...,...,...,...,...,...,...
178,178,2.128380,8.81758,1.298890,2.884440,2.134380,3,3,5,1
179,179,0.991895,9.58108,0.249952,3.514370,-1.466080,3,3,1,1
180,180,2.925110,8.49961,0.318501,-2.456960,-2.902510,4,4,8,2
181,181,0.738100,9.87662,1.041450,2.984240,-2.138320,3,3,1,1


In [5]:
import pyvista as pv
pv.global_theme.color = 'white'

# Different color scheme for different type of labels


# For Metadata
# new_colors = ["#b18281", "#6d45cd", "#62a03b", "#c84ccb", "#a68b3c", "#482a79","#d74327", 
#              "#6f7dcf", "#cf783d", "#608aa4", "#cd4859", "#5f9c7b", "#d2478d", "#44532d", 
#              "#b773b5", "#703425", "#342d40", "#723057"]

# # For Topology
# new_colors = ["#dca083", "#7dbad3", "#983aa1", "#b1c232", "#db4393", "#a5662f", "#733fd4",
#               "#24192f", "#3d2560", "#4d8163", "#64c27a", "#a9ad84", "#86af46", "#db502c",
#               "#882d2a", "#50222f", "#d64ed7", "#c4a2c5", "#34475e", "#5a3d23", "#e19a2e",
#               "#bea44b", "#557f8f", "#d8405a", "#462b8d", "#6a6d2b", "#5ec0ab", "#2b3f28",
#               "#648ace", "#d67681", "#54c840", "#397c30", "#8f6c64", "#903767", "#d082cd",
#               "#715c8b", "#7070dc"]

# # For Features
new_colors = ["#b8934e", "#c44f39", "#819bb1", "#4d393d", "#69aa55", "#c65b94", "#7d4cba"]


def get_spherical_cap_structure_grid(b, opening_angle, R, color_idx, radius=1.0):
    # From: https://stackoverflow.com/a/45458451
    r = R
    phi = np.linspace(0, 2 * np.pi, 30)
    theta = np.linspace(0, opening_angle, 20)
    X = r * np.stack([
        np.outer(np.cos(phi), np.sin(theta)),
        np.outer(np.sin(phi), np.sin(theta)),
        np.outer(np.ones(np.size(phi)), np.cos(theta)),
        ], axis=-1)

    # rotate X such that [0, 0, 1] gets rotated to `c`;
    # <https://math.stackexchange.com/a/476311/36678>.
    a = np.array([0.0, 0.0, 1.0])
    a_x_b = np.cross(a, b)
    a_dot_b = np.dot(a, b)
    if a_dot_b == -1.0:
        X_rot = -X
    else:
        X_rot = (
            X +
            np.cross(a_x_b, X) +
            np.cross(a_x_b, np.cross(a_x_b, X)) / (1.0 + a_dot_b)
            )
        
    return pv.StructuredGrid(X_rot[..., 0], X_rot[..., 1], X_rot[..., 2])

In [6]:
def get_geodesic(p1, p2):
    omega = np.arccos(np.dot(p1, p2) / (np.linalg.norm(p1) * np.linalg.norm(p2)))
    t = np.linspace(0, 1)
    
    line = []
    for t in np.linspace(0, 1):
        line.append(np.sin((1 - t) * omega) / np.sin(omega) * p1 + np.sin(t * omega) / np.sin(omega) * p2)
    return np.array(line)

In [7]:
def compute_prob_S2(p1, p2, kappa1, kappa2):
    beta = 2.24
    mu = 0.0123
    R = 1
    angle = np.arccos(np.dot(p1, p2) / (np.linalg.norm(p1) * np.linalg.norm(p2)))
    
    chi = (R * angle) / np.sqrt(kappa1 * kappa2 * mu)
    return 1 / (1 + np.power(chi, beta))

In [8]:
def plot_embedding(df, label):
    pv.set_plot_theme("document")
    plotter = pv.Plotter(window_size=[4096, 4096])

    plotter.enable_anti_aliasing('ssaa')

    R = 1
    u, v = np.mgrid[0:2*np.pi:100j, 0:np.pi:100j]
    x = R*np.cos(u)*np.sin(v)
    y = R*np.sin(u)*np.sin(v)
    z = R*np.cos(v)
    grid = pv.StructuredGrid(x, y, z)
    plotter.add_mesh(grid, color='#fdfdfd', opacity=1)

    # Plot edges
    pos = df[['p1', 'p2', 'p3']].values
    pos /= np.linalg.norm(pos, axis=1)[:, None]
    kappa = df['kappa'].values

    count = 0
    for source, target in tqdm(texas_edges.values):
        s_i = df['index'].tolist().index(source)
        t_i = df['index'].tolist().index(target)

        # Compute the probability of connection
        p1, p2 = pos[s_i], pos[t_i]
        prob = compute_prob_S2(p1, p2, kappa[s_i], kappa[t_i])
        if prob < 0.5: # filter out low probable links
            count += 1
            continue

        l = get_geodesic(p1, p2)
        actor = plotter.add_lines(l, color='#8a8a8a', width=6*prob)

    print('Number of low probable links: ', count)

    max_kappa = max(df['kappa'].values)
    idx = 0
    i = 0
    R = 1.001
    for name, group in df.groupby(label):

        pos = group[['p1', 'p2', 'p3']].values
        for j in range(len(group)):
            p = pos[j] / np.linalg.norm(pos[j])
            s = group['kappa'].values[j]
            s /= max_kappa
            s = np.log(s + 1.5)
            s *= 0.10
            
            cap = get_spherical_cap_structure_grid(p, s, R, color_idx=idx)
            plotter.add_mesh(cap, color=new_colors[idx])
            i += 1
        idx += 1


    plotter.camera_position = 'yz'

    # LE+ML
    plotter.camera.azimuth = 20
    plotter.camera.elevation = 10
    
    # UMAP+ML
    #plotter.camera.azimuth = 20
    #plotter.camera.elevation = 30
    
    return plotter

In [9]:
pv.start_xvfb()

In [10]:
# plotter = plot_embedding(texas_le_ml, 'label')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/texas_le_ml_labels_metadata.jpg")


In [11]:
# plotter = plot_embedding(texas_umap_ml, 'label')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/texas_umap_ml_labels_metadata.jpg")


In [12]:
# plotter = plot_embedding(texas_only_umap, 'label')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/texas_only_umap_labels_metadata.jpg")


In [13]:
# plotter = plot_embedding(texas_le_ml, 'label_louvain')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/texas_le_ml_labels_topology.jpg")


In [14]:
# plotter = plot_embedding(texas_umap_ml, 'label_louvain')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/texas_umap_ml_labels_topology.jpg")


In [15]:
# plotter = plot_embedding(texas_only_umap, 'label_louvain')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/texas_only_umap_labels_topology.jpg")


In [16]:
# plotter = plot_embedding(texas_le_ml, 'label_clustering_find_k')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/texas_le_ml_labels_features.jpg")


In [17]:
# plotter = plot_embedding(texas_umap_ml, 'label_clustering_find_k')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/texas_umap_ml_labels_features.jpg")


In [18]:
# plotter = plot_embedding(texas_only_umap, 'label_clustering_find_k')
# plotter.screenshot("/home/rob/Dropbox/NodesFeaturesEmbeddings/Report/figures-publication-and-random-initialization-27-04-23/plots/texas_only_umap_labels_features.jpg")


  0%|          | 0/295 [00:00<?, ?it/s]

Number of low probable links:  127


array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]